Upload the dataset


In [98]:
import pandas as pd

# Upload the dataset
df = pd.read_csv(r'C:\Users\peppe\Desktop\GoodReads Recommender System\Dataset\user_rating_0_to_1000.csv')
df.head()

,ID,Name,Rating
0,1,Agile Web Development with Rails: A Pragmatic ...,it was amazing
1,1,The Restaurant at the End of the Universe (Hit...,it was amazing
2,1,Siddhartha,it was amazing
3,1,The Clock of the Long Now: Time and Responsibi...,really liked it
4,1,"Ready Player One (Ready Player One, #1)",really liked it


Rename column names

In [99]:
df.rename(columns={'ID': 'user_id', 'Name': 'title', 'Rating': 'rating'}, inplace=True)
df.head()

,user_id,title,rating
0,1,Agile Web Development with Rails: A Pragmatic ...,it was amazing
1,1,The Restaurant at the End of the Universe (Hit...,it was amazing
2,1,Siddhartha,it was amazing
3,1,The Clock of the Long Now: Time and Responsibi...,really liked it
4,1,"Ready Player One (Ready Player One, #1)",really liked it


Change the Ratings from text to 0-5

In [100]:
rating_mapping = {
    'did not like it': 1,
    'it was ok': 2,
    'liked it': 3,
    'really liked it': 4,
    'it was amazing': 5,
    "This user doesn't have any rating": 0
}

df['rating'] = df['rating'].replace(rating_mapping)
df.head(10)

,user_id,title,rating
0,1,Agile Web Development with Rails: A Pragmatic ...,5
1,1,The Restaurant at the End of the Universe (Hit...,5
2,1,Siddhartha,5
3,1,The Clock of the Long Now: Time and Responsibi...,4
4,1,"Ready Player One (Ready Player One, #1)",4
5,1,"The Hunger Games (The Hunger Games, #1)",5
6,1,"The Clue in the Embers (Hardy Boys, #35)",5
7,1,The Authoritative Calvin and Hobbes: A Calvin ...,5
8,1,"The Clue of the Broken Blade (Hardy Boys, #21)",5
9,1,"The Clue of the Hissing Serpent (Hardy Boys, #53)",5


Create the ratings matrix associated with the DataFrame by taking the user identifiers as an index and the book titles as columns, storing the matrix in a variable named mat_ratings. Replace the missing values ​​with 0s.

In [101]:
# Ensure the rating column is numeric
df['rating'] = pd.to_numeric(df['rating'], errors='coerce')

# Create the ratings matrix
mat_ratings = df.pivot_table(columns='title', index='user_id', values='rating')



In [102]:
mat_ratings = mat_ratings.fillna(0)


Create the CSR matrix sparse_ratings from the matrix mat_ratings. We will also store user IDs and book titles in variables named user_ids and titles, respectively.

In [103]:
from scipy.sparse import csr_matrix

sparse_ratings = csr_matrix(mat_ratings)

user_ids = mat_ratings.index.tolist()
titles = mat_ratings.columns.tolist()

 The TruncatedSVD method allows us to do Truncated Singular Value Decomposition. The n_components parameter allows us to choose the number of singular values ​​(or latent factors) to keep. Create an instance of TruncatedSVD, 0 svd with n_components = 12. Then apply the transformation to sparse_ratings.T in a variable that we will name ratings_red* . Show the dimensions of *ratings_red.

In [104]:

from sklearn.decomposition import TruncatedSVD

# Create an instance of TruncatedSVD with n_components = 12
svd = TruncatedSVD(n_components=12)

# Apply the transformation to sparse_ratings.T
ratings_red = svd.fit_transform(sparse_ratings.T)

# Show the dimensions of ratings_red
ratings_red.shape


(24094, 12)

Reconstruct the original matrix using the inverse_transform method and store it in a variable named new_ratings_red.

(f) Create the item_similarity and item_similarity2 matrices using the ratings_red and new_ratings_red matrices, respectively. We will use the cosine_similarity function. Then convert the matrices to DataFrame. To ensure that we have the right books corresponding to each similarity, we will use the titles_books variable previously defined for the index and columns parameters. As a reminder, it contains the user identifiers in the order of the rating matrix.

In [105]:
from sklearn.metrics.pairwise import cosine_similarity

# Reconstruct the original matrix
new_ratings_red = svd.inverse_transform(ratings_red)

# Create the item similarity matrices
item_similarity = cosine_similarity(ratings_red)
item_similarity2 = cosine_similarity(new_ratings_red)

# Convert the matrices to DataFrame
item_similarity_df = pd.DataFrame(item_similarity, index=titles, columns=titles)
item_similarity2_df = pd.DataFrame(item_similarity2, index=titles, columns=titles)



In [106]:
import numpy as np
def pred_item(mat_ratings, item_similarity, k, user_id):

    # Select in mat_ratings the books that have not yet been read by the user
    to_predict = mat_ratings.loc[user_id][mat_ratings.loc[user_id]==0]
    
    # Iterate over all these books
    for i in to_predict.index:

        #Find the k most similar books excluding the book itself
        similar_items = item_similarity.loc[i].sort_values(ascending=False)[1:k+1]

        # Calculation of the norm of the similar_items vector
        norm = np.sum(np.abs(similar_items))

        # Retrieve the ratings given by the user to the k nearest neighbors
        ratings = mat_ratings[similar_items.index].loc[user_id]

        # Calculate the dot product between ratings and similar_items
        scalar_prod = np.dot(ratings, similar_items)
        
        #Calculate predicted rating for movie i
        pred = scalar_prod / norm

        # Replace with prediction
        to_predict[i] = pred


    return to_predict

 Display the 10 best ratings above 4 from the user '5' which will be stored in the variable user_preferences.

In [107]:
userId = 5

# Ratings above 4
user_preferences = df[(df['user_id'] == userId) & (df['rating'] >= 4)]

# We sort in descending order, taking the top 10
user_preferences = user_preferences.sort_values('rating', ascending=False).head(10)

user_preferences


,user_id,title,rating
580,5,The Secret Garden,5
846,5,The Life-Changing Magic of Tidying Up: The Jap...,5
1010,5,Inspired Styles,5
1007,5,The Great Gatsby,5
1004,5,"Anne of Green Gables (Anne of Green Gables, #1)",5
1003,5,Far To Go,5
1001,5,West with the Night,5
999,5,"Family Shoes (Shoes, #8)",5
998,5,Slim Aarons: Women,5
995,5,The Beauty Myth,5


 Make 10 recommendations to user '5' using the pred_item function defined previously and using item_similarity and item_similarity2. Compare the results obtained.

In [108]:
reco_user1 = pred_item(mat_ratings, item_similarity_df, 3, 5).sort_values(ascending=False).head(10)
print(reco_user1)


C:\Users\peppe\AppData\Local\Temp\ipykernel_45392\4012399455.py:23: RuntimeWarning: invalid value encountered in scalar divide
  pred = scalar_prod / norm


title
The Gifts of Imperfection                                                      4.659184
Money                                                                          4.333898
Wuthering Heights                                                              2.659972
Catching Fire (The Hunger Games, #2)                                           2.000797
Slaughterhouse-Five                                                            1.674954
Banker to the Poor: Micro-Lending and the Battle Against World Poverty         1.674227
Rebecca                                                                        1.673997
The Alchemyst (The Secrets of the Immortal Nicholas Flamel, #1)                1.669648
The Hitchhiker's Guide to the Galaxy (Hitchhiker's Guide to the Galaxy, #1)    1.668967
The Guernsey Literary and Potato Peel Pie Society                              1.665995
Name: 5, dtype: float64


In [109]:

reco_user1bis = pred_item(mat_ratings, item_similarity2_df, 3, 5).sort_values(ascending=False).head(10)
print(reco_user1bis)

C:\Users\peppe\AppData\Local\Temp\ipykernel_45392\4012399455.py:23: RuntimeWarning: invalid value encountered in scalar divide
  pred = scalar_prod / norm


title
The Gifts of Imperfection                                                      4.659184
Money                                                                          4.333898
Wuthering Heights                                                              2.659972
Catching Fire (The Hunger Games, #2)                                           2.000797
Slaughterhouse-Five                                                            1.674954
Banker to the Poor: Micro-Lending and the Battle Against World Poverty         1.674227
Rebecca                                                                        1.673997
The Alchemyst (The Secrets of the Immortal Nicholas Flamel, #1)                1.669648
The Hitchhiker's Guide to the Galaxy (Hitchhiker's Guide to the Galaxy, #1)    1.668967
The Guernsey Literary and Potato Peel Pie Society                              1.665995
Name: 5, dtype: float64


In both cases we can note that the recommendations do not change if we reconstruct our original matrix than if we take the reduced matrix. However, the recommendations will change depending on the number of factors that are taken into account. How to make the right choice of parameters? We will explore this later on.

From the Reader function, create an object that we will name reader by putting rating_scale=(0, 5) as parameter. Then, read the database using the Dataset.load_from_df function and storing it in a variable that we will name df_surprise. For parameters we will use df[["user_id", "title", "rating"]] and reader=reader.

In [110]:
from surprise import Reader
from surprise import Dataset

reader = Reader(rating_scale=(0, 5))

df_surprise = Dataset.load_from_df(df[["user_id", "title", "rating"]], reader=reader)

In [111]:
user_id = 5
from surprise import NormalPredictor
from surprise.model_selection import cross_validate

normpred = NormalPredictor()

cross_validate(normpred, df_surprise, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm NormalPredictor on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.3273  1.3243  1.3293  1.3305  1.3292  1.3281  0.0022  
MAE (testset)     1.0514  1.0475  1.0474  1.0561  1.0545  1.0514  0.0035  
Fit time          0.02    0.03    0.02    0.02    0.02    0.02    0.00    
Test time         0.02    0.02    0.02    0.02    0.02    0.02    0.00    


{'test_rmse': array([1.3273471 , 1.32427263, 1.32929163, 1.33046009, 1.32920087]),
 'test_mae': array([1.05142827, 1.04747318, 1.04738371, 1.05607751, 1.05446988]),
 'fit_time': (0.023508548736572266,
  0.02501678466796875,
  0.021029949188232422,
  0.02200031280517578,
  0.02312779426574707),
 'test_time': (0.01999211311340332,
  0.022505998611450195,
  0.022511959075927734,
  0.023014068603515625,
  0.02300119400024414)}

Let's see the predictions this algorithm gives for our user. We will first create our test set using the build_full_trainset(). function.

Traditionally, in Machine Learning we evaluate our model by separating our database into two: a part for training and a part for testing. Except that, in Recommendation Systems this implies that our sample of Training and our validation sample correspond to a different user population, which doesn't make much sense.

Here what we are going to do is to construct a train set and then a test set containing the pairs of notations not present in our train set. This can be done easily using the function build_anti_testset().

In our case, to avoid having to iterate on all users and all books, we will only focus on our user's information to create the antitest set.

We also need to assign a rating value to the unrated pairs, so we will use the overall average of the training set (which is the default value used by Surprise).

(d) Build train_set using the function build_full_trainset().

(e) From the train set, extract the id used by Surprise associated with our user. To do this, use the function to_inner_uid(user_id). Then store it in a variable named targetUser.

(f) In a list named anti_testset_user fill in the books not rated by our user with the overall average for rating.

In [112]:
# Build train_set using the function build_full_trainset()
train_set = df_surprise.build_full_trainset()

# Extract the id used by Surprise associated with our user
targetUser = train_set.to_inner_uid(user_id)

# Create the anti-test set for the user
anti_testset = []
rated_items = [item for (item, _) in train_set.ur[targetUser]]
for item_id in train_set.all_items():
    if item_id not in rated_items:
        anti_testset.append((targetUser, item_id, train_set.global_mean))


In a variable predictions store the predictions of our algorithm normpred. Convert it to a DataFrame, sort by 'est' in a descending manner and display the first ten recommendations. In Surprise 'est' is the predicted note.

In [116]:
# Solution

# Perform predictions on the "anti-testset_user" set using the model (not specified in the code)
predictions = normpred.test(anti_testset)  

# Convert predictions to a pandas DataFrame
predictions = pd.DataFrame(predictions)

# Create a mapping from numerical ID back to original titles
title_mapping = dict(enumerate(df["title"].astype("category").cat.categories))

# Apply mapping to convert numerical iid back to book titles
predictions["iid"] = predictions["iid"].map(title_mapping)

# Sort predictions by column 'est' (estimate) in descending order
predictions.sort_values(by=['est'], inplace=True, ascending=False)

# Show top 10 predictions
predictions.head(10)

,uid,iid,r_ui,est,details
17219,4,The Good Girl,3.82287,5.0,{'was_impossible': False}
2638,4,Childe Roland to the Dark Tower Came,3.82287,5.0,{'was_impossible': False}
7501,4,If The Creek Don't Rise,3.82287,5.0,{'was_impossible': False}
2603,4,Chemistry,3.82287,5.0,{'was_impossible': False}
2606,4,Chesapeake,3.82287,5.0,{'was_impossible': False}
15037,4,"The Anome (Durdane, #1)",3.82287,5.0,{'was_impossible': False}
15044,4,The Antimonopoly Laws Of Japan,3.82287,5.0,{'was_impossible': False}
7490,4,Idoru (Bridge #2),3.82287,5.0,{'was_impossible': False}
7487,4,Identical Strangers: A Memoir of Twins Separat...,3.82287,5.0,{'was_impossible': False}
11481,4,Opening the Bidding,3.82287,5.0,{'was_impossible': False}


In [117]:
from surprise import SVD
svd = SVD()

cross_validate(svd, df_surprise, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8671  0.8685  0.8662  0.8630  0.8631  0.8656  0.0022  
MAE (testset)     0.6833  0.6849  0.6856  0.6827  0.6843  0.6842  0.0011  
Fit time          0.31    0.31    0.32    0.32    0.30    0.31    0.01    
Test time         0.03    0.03    0.03    0.03    0.03    0.03    0.00    


{'test_rmse': array([0.86706626, 0.86852546, 0.86623589, 0.86301346, 0.86307654]),
 'test_mae': array([0.68328812, 0.68492957, 0.68562944, 0.68270685, 0.68425197]),
 'fit_time': (0.31074976921081543,
  0.3091449737548828,
  0.3210155963897705,
  0.3204183578491211,
  0.30475687980651855),
 'test_time': (0.02653193473815918,
  0.02626347541809082,
  0.026000022888183594,
  0.0260009765625,
  0.027999401092529297)}

In [118]:
# Perform predictions on the "anti-testset_user" set using the model (not specified in the code)
predictionsSVD = svd.test(anti_testset)  

# Convert predictions to a pandas DataFrame
predictionsSVD = pd.DataFrame(predictionsSVD)

# Sort predictions by column 'is' (estimate) in descending order
predictionsSVD.sort_values(by=['est'], inplace=True, ascending=False)

# Show top 10 predictions
predictionsSVD.tail(10)

,uid,iid,r_ui,est,details
7791,4,8494,3.82287,4.042496,{'was_impossible': False}
7799,4,8502,3.82287,4.042496,{'was_impossible': False}
7798,4,8501,3.82287,4.042496,{'was_impossible': False}
7797,4,8500,3.82287,4.042496,{'was_impossible': False}
7796,4,8499,3.82287,4.042496,{'was_impossible': False}
7795,4,8498,3.82287,4.042496,{'was_impossible': False}
7794,4,8497,3.82287,4.042496,{'was_impossible': False}
7793,4,8496,3.82287,4.042496,{'was_impossible': False}
7792,4,8495,3.82287,4.042496,{'was_impossible': False}
23390,4,24093,3.82287,4.042496,{'was_impossible': False}


Perform Gridsearch for SVD

In [132]:
from surprise.model_selection import GridSearchCV

param_grid = {'n_factors': [100, 150],
              'n_epochs': [20, 25, 30],
              'lr_all':[0.005, 0.01, 0.1],
              'reg_all':[0.02, 0.05, 0.1]}
grid_search = GridSearchCV(SVD, param_grid, measures=['rmse','mae'], cv=3)
grid_search.fit(df_surprise)

In [133]:

print(grid_search.best_score['rmse'])

print(grid_search.best_score['mae'])

print(grid_search.best_params['rmse'])

tunedSVD = grid_search.best_estimator['rmse']

cross_validate(tunedSVD, df_surprise, measures=['RMSE', 'MAE'], cv=5, verbose=True)

0.8618462358744828
0.6836813938590174
{'n_factors': 100, 'n_epochs': 30, 'lr_all': 0.005, 'reg_all': 0.1}
Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8572  0.8574  0.8608  0.8545  0.8674  0.8595  0.0045  
MAE (testset)     0.6789  0.6839  0.6789  0.6769  0.6913  0.6820  0.0052  
Fit time          0.43    0.43    0.44    0.42    0.44    0.43    0.01    
Test time         0.03    0.03    0.03    0.03    0.03    0.03    0.00    


{'test_rmse': array([0.85719339, 0.85739796, 0.8607743 , 0.85452654, 0.86744229]),
 'test_mae': array([0.6789361 , 0.68386357, 0.67893213, 0.67692049, 0.69133291]),
 'fit_time': (0.4338717460632324,
  0.4304616451263428,
  0.43692708015441895,
  0.4164571762084961,
  0.4433131217956543),
 'test_time': (0.025290250778198242,
  0.025766372680664062,
  0.026013612747192383,
  0.02515244483947754,
  0.026001453399658203)}

In [134]:
from surprise import dump

# Save the best SVD model
dump.dump('svd_model', algo=tunedSVD)

Perform Gridsearch for NMF

In [130]:
from surprise.model_selection import GridSearchCV
from surprise import NMF

param_grid = {'n_factors': [18, 100],
              'n_epochs': [20],
              'reg_pu': [0.02, 0.05]}

# Initialize the GridSearchCV
grid_search_2 = GridSearchCV(NMF, param_grid, measures=['rmse', 'mae'], cv=3)

# Fit the model
grid_search_2.fit(df_surprise)


In [131]:

print(grid_search_2.best_score['rmse'])

print(grid_search_2.best_score['mae'])

print(grid_search_2.best_params['rmse'])

tunedNMF = grid_search_2.best_estimator['rmse']

cross_validate(tunedNMF, df_surprise, measures=['RMSE', 'MAE'], cv=5, verbose=True)

1.019342445140591
0.7787451809812712
{'n_factors': 18, 'n_epochs': 20, 'reg_pu': 0.05}
Evaluating RMSE, MAE of algorithm NMF on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.0010  1.0231  1.0289  1.0075  1.0141  1.0149  0.0101  
MAE (testset)     0.7691  0.7800  0.7852  0.7686  0.7727  0.7751  0.0065  
Fit time          0.42    0.42    0.42    0.41    0.42    0.42    0.00    
Test time         0.02    0.02    0.02    0.03    0.02    0.02    0.00    


{'test_rmse': array([1.00095529, 1.02309989, 1.02886129, 1.00750894, 1.01414839]),
 'test_mae': array([0.76908215, 0.77998114, 0.78517548, 0.76861896, 0.77272312]),
 'fit_time': (0.420072078704834,
  0.4210538864135742,
  0.4185209274291992,
  0.4125678539276123,
  0.4179694652557373),
 'test_time': (0.023230314254760742,
  0.024919986724853516,
  0.023998022079467773,
  0.025275230407714844,
  0.024130821228027344)}